<a href="https://colab.research.google.com/github/Wxtar777/codealpha_basic_chatbot/blob/main/basice_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle
import random
import nltk
import json
from nltk.stem.lancaster import LancasterStemmer

In [ ]:
lematizer = nltk.stem.WordNetLemmatizer()

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open('intents.json') as json_data:
    intents1 = json.load(json_data)

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents1['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lematizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [ ]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0] # targets
    pattern_words = [lematizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

#random.shuffle(training)

training = np.array(training, dtype="object")

x_train = list(training[:,0])
y_train = list(training[:,1])
x_train = np.array(x_train)
y_train = np.array(y_train)

# training

In [ ]:
input_shape = (len(x_train[0]),)
output_shape = len(y_train[0])

# Create the model
model = Sequential()
model.add(Dense(16, input_shape=input_shape, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(output_shape, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(x_train, y_train, epochs=300)

In [ ]:
model.save('chatbot_model.keras', hist)

# building the chatbot

In [ ]:
nltk.download('wordnet')

In [ ]:
lem = nltk.stem.WordNetLemmatizer()

In [ ]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = tf.keras.models.load_model('chatbot_model.keras')

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
def clean_sentence(sentence):
    tokenized_sentence = nltk.word_tokenize(sentence)
    post_taged_sentence = nltk.pos_tag(tokenized_sentence)
    output = []
    for word, tag in post_taged_sentence:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
             lemma = word
        else:
             lemma = lem.lemmatize(word, wntag)
        output.append(lemma)
    return output


In [ ]:
def bag_of_words(sentence):
    sentence = clean_sentence(sentence)
    bag = [0] * len(words)

    for i in sentence:
        for j, word in enumerate(words):
            if word == i:
                bag[j] = 1
    return bag

In [ ]:
def predict(sentence):
   sentence_bag_of_words = bag_of_words(sentence)
   prediction = model.predict(np.array([sentence_bag_of_words]))[0]

   result = [[i, r] for i, r in enumerate(prediction) if r > 0.25]

   result.sort(key=lambda x: x[1], reverse=True)
   return_list = []
   for r in result:
       return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
   return return_list

In [ ]:
def response(intents, intents_json):
    tag = intents[0]['intent']
    list_of_intents = intents_json['intents']

    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
while True:
    inp = input("You: ")
    if inp.lower() == "quit":
        break

    result = predict(inp)
    print("Bot: ", end="")
    print(response(result, intents))
    if result[0]['intent'] == 'goodbye':
        break


In [ ]:
predict("hi")